In [1]:
import pickle as pkl
import pandas as pd

In [2]:
loan = pd.read_csv('loan_borowwer_data.csv')

In [3]:
dt = pkl.load(open('decision_tree.pkl', 'rb'))
minmax = pkl.load(open('minmax_scaler.pkl', 'rb'))
lb = pkl.load(open('label_binarizer.pkl', 'rb'))

In [4]:
loan.shape

(9578, 14)

In [5]:
lb.classes_

array(['all_other', 'credit_card', 'debt_consolidation', 'educational',
       'home_improvement', 'major_purchase', 'small_business'],
      dtype='<U18')

In [6]:
loan_purpose_dummy = lb.transform(loan['purpose'])
loan_purpose_df = pd.DataFrame(loan_purpose_dummy, columns=lb.classes_)
loan_with_purpose_dummy = loan.join(loan_purpose_df)
loan_final = loan_with_purpose_dummy.drop(['purpose'], axis=1)

In [7]:
x = loan_final.drop('not.fully.paid', axis=1)
y = loan_final['not.fully.paid']

In [8]:
x.shape

(9578, 19)

In [9]:
x_transform = minmax.transform(x)

In [10]:
loan['predict'] = dt.predict(x_transform)

### Metrics

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
confusion_matrix(y, loan.predict)

array([[7963,   82],
       [1164,  369]], dtype=int64)

In [13]:
print(classification_report(y, loan.predict))

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      8045
           1       0.82      0.24      0.37      1533

    accuracy                           0.87      9578
   macro avg       0.85      0.62      0.65      9578
weighted avg       0.86      0.87      0.84      9578



## Single Row Prediction

In [14]:
import json

In [15]:
data = loan.drop(['not.fully.paid'], axis=1).iloc[0].to_json()

In [16]:
data

'{"credit.policy":1,"purpose":"debt_consolidation","int.rate":0.1189,"installment":829.1,"log.annual.inc":11.35040654,"dti":19.48,"fico":737,"days.with.cr.line":5639.958333,"revol.bal":28854,"revol.util":52.1,"inq.last.6mths":0,"delinq.2yrs":0,"pub.rec":0,"predict":0}'

In [17]:
def data_transfrom(data):
    loan = pd.DataFrame.from_dict([json.loads(data)])

    loan_purpose_dummy = lb.transform(loan['purpose'])
    loan_purpose_df = pd.DataFrame(loan_purpose_dummy, columns=lb.classes_)
    loan_with_purpose_dummy = loan.join(loan_purpose_df)
    loan_final = loan_with_purpose_dummy.drop(['purpose'], axis=1)#    return x_transform

    x_transform = minmax.transform(loan_final)

    return x_transform

In [18]:
x_transform = data_transfrom(data)

ValueError: X has 20 features, but MinMaxScaler is expecting 19 features as input.

In [ ]:
dt.predict(x_transform)

## Check Service

In [19]:
import requests

In [20]:
data = '{"credit.policy":1,"purpose":"debt_consolidation","int.rate":0.1189,"installment":829.1,"log.annual.inc":11.35040654,"dti":19.48,"fico":737,"days.with.cr.line":5639.958333,"revol.bal":28854,"revol.util":52.1,"inq.last.6mths":0,"delinq.2yrs":0,"pub.rec":0}'

In [21]:
requests.post('http://127.0.0.1:5000/model_predict', json={'data': data}).text

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /model_predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EAA572C7F0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))